In this notebook, we're going to talk about [`pandas`](https://pandas.pydata.org/) — a popular Python library for working specifically with tabular data. It is centered around two different data structures: ***Series*** and ***DataFrames***. `pandas` relies heavily on `numpy` to make operations on these data structures fast.

In [ ]:
!pip install pandas
import pandas as pd

# Important: Run this code cell each time you start a new session!

In [ ]:
!pip install os
import os
import numpy as np
import pandas as pd

In [ ]:
!wget -nc https://archive.physionet.org/users/shared/challenge-2019/training_setA.zip
!unzip -n training_setA.zip

In [ ]:
def load_single_file(file_path):
    df = pd.read_csv(file_path, sep="|")
    df['PatientID'] = file_path.split(os.sep)[-1][:-4]
    df['Hour'] = df.index
    keep_cols = ['PatientID', 'Age', 'Gender', 'SepsisLabel', 'Hour',
                 'HR', 'O2Sat', 'SBP', 'DBP', 'Resp']
    df = df[keep_cols]
    df.rename(columns={'Gender': 'Sex'}, inplace=True)
    return df

def create_final_table():
    final_df = pd.DataFrame()
    for f in os.listdir("training")[:10]:
        df = load_single_file(os.path.join("training", f))
        final_df = pd.concat([final_df, df])
    final_df.to_csv('sepsis.csv',index=False)
create_final_table()

# Creating Series

A `Series` is essentially a 1D column array that can hold data of any type:



In [ ]:
pd.Series([67, 77, 75, 83, 64])

Notice that when we print out the `Series`, we actually see two columns of numbers. On the right, we see the numbers we used to initialize the `Series`. The numbers on the left are the labels we can use to access each element. `pandas` calls these labels the ***index*** of the `Series`, but bear in mind that they are different from the positional indices you've used for lists and arrays.

By default, `pandas` automatically generates indices corresponding to the position of the element in the `Series`, but you can give your `Series` more informative indices as you see fit. For example, recall the patient data we looked at earlier:

In [ ]:
name_data = ["Alice", "Bob", "Carol", "Dan", "Eric"]
sex_data = ["female", "male", "female", "male", "male"]
age_data = [67, 77, 75, 83, 64]

We can turn `sexes` and `ages` into their own `Series` and then index those elements by the patients' names:

In [ ]:
sexes = pd.Series(sex_data, index=name_data)
sexes

In [ ]:
ages = pd.Series(age_data, index=name_data)
ages

As a side note, we can also create a `Series` with indices using a `dict`:

In [ ]:
# Creating a Series using a dictionary
pd.Series({"Alice": 67, "Bob": 77, "Carol": 75, "Dan": 83, "Eric": 64})

It is strongly suggested that you keep your indices unique. Otherwise, you can run into weird issues as you merge, join, or resample your data.

# Accessing Series Elements

If we want to grab elements within a `Series`, we can access them by their assigned label using `.loc[]`. In our toy dataset, we use the patients' names as the labeling index.

In [ ]:
name_data = ["Alice", "Bob", "Carol", "Dan", "Eric"]
sex_data = ["female", "male", "female", "male", "male"]
age_data = [67, 77, 75, 83, 64]
ages = pd.Series(age_data, index=name_data)

In [ ]:
# Grab Alice and Bob's ages
ages.loc[['Alice', 'Bob']]

If you want this data to look like a `numpy` array, you can simple look at the `.values` attribute.

In [ ]:
ages.loc[['Alice', 'Bob']].values

If you did not create an index for your `Series`, remember that `pandas` will create numerical indices for you. When this happens, it will ***seem like*** you can access rows by their position using `loc[]`.

In [ ]:
ages_noindex = pd.Series(age_data)
ages_noindex

In [ ]:
# Grab the rows with the assigned labels 0 thru 4
ages_noindex.loc[:2]

However, you'll find that this is not actually the case because you won't be able to use negative indices.

In [ ]:
 # Grab the row with the label -1, which doesn't exist
ages_noindex.loc[-1]

If you really want to access values by their position in the series, you can use the `.iloc[]` operator.

In [ ]:
# Grab the last row
ages_noindex.iloc[-1]

That being said, using the labels built into the `Series` will give you more power in the long run.

# Creating DataFrames

A `DataFrame` is a 2D array composed of multiple `Series`, which finally brings us to a structure that closely resembles tabular data. This is just one of many ways to create a `DataFrame`; you can also create them from lists, dictionaries, etc.

In [ ]:
names = ["Alice", "Bob", "Carol", "Dan", "Eric"]
sexes = ["female", "male", "female", "male", "male"]
ages = [67, 77, 75, 83, 64]

sex_series = pd.Series(sexes, index=names)
age_series = pd.Series(ages, index=names)
patients_df = pd.DataFrame({'Sex': sex_series, 'Age': age_series})
patients_df

,Sex,Age
Alice,female,67
Bob,male,77
Carol,female,75
Dan,male,83
Eric,male,64


Chances are that you won't be creating `DataFrames` from scratch. Instead, you'll be loading data from a file. Let's take a look at a real-world dataset from the [2019 PhysioNet Challenge](https://physionet.org/content/challenge-2019/1.0.0/), which was aimed at predicting sepsis from physiological data. Note that we have saved an extremely condensed version of this dataset in a single `.csv`.

In [ ]:
df = pd.read_csv("sepsis.csv")
df

,PatientID,Age,Sex,SepsisLabel,Hour,HR,O2Sat,SBP,DBP,Resp
0,p007806,51.86,0,0,0,NaN,NaN,NaN,NaN,NaN
1,p007806,51.86,0,0,1,70.0,98.0,106.0,NaN,15.0
2,p007806,51.86,0,0,2,NaN,NaN,NaN,NaN,NaN
3,p007806,51.86,0,0,3,63.0,97.0,104.0,NaN,19.0
4,p007806,51.86,0,0,4,74.0,99.0,107.0,NaN,20.0
...,...,...,...,...,...,...,...,...,...,...
371,p005568,76.94,0,0,36,NaN,NaN,NaN,NaN,NaN
372,p005568,76.94,0,0,37,78.0,96.0,147.0,NaN,10.0
373,p005568,76.94,0,0,38,NaN,NaN,NaN,NaN,NaN
374,p005568,76.94,0,0,39,78.0,95.0,125.0,NaN,15.0


All we did was give this function a filename, and `pandas` was automatically able to do the following for us:
* Assume that the first row was the header of the `.csv` file and therefore named the columns accordingly
* Assume the type of data in each column based on their syntax and make the conversions for us
* Fill in missing data with placeholders (`np.nan`)

This is a really powerful function with over 20 different optional arguments that you can investigate on your own.

We will play with this `DataFrame` for the rest of this notebook.

# Accessing DataFrame Columns



We can grab one or many columns by indexing into the `DataFrame` according to the column name(s):

In [ ]:
df['PatientID'] # Provides a Series

In [ ]:
df[['PatientID', 'Age']] # Provides a DataFrame

# Accessing DataFrame Rows

We can grab one or many rows within a `DataFrame` using either the `.loc[]`
or `.iloc[]` operator that we used earlier for `Series`.

In [ ]:
# Grab the rows with the assigned labels 0 thru 4
df.loc[0:4]

In [ ]:
# Grab the last row
df.iloc[-1]

Where `pandas` becomes very powerful is the fact that you can grab rows that satisfy a set of criteria specified as boolean statements. Here are some examples:

In [ ]:
# Grab all the rows from male patients
df[df['Sex'] == 1]

In [ ]:
# Grab all the rows with heart rate values greater than 100
df[df['HR'] > 100]

In [ ]:
# Grab all of the rows between the hours of 7 AM to 8 AM
df[(df['Hour'] >= 7) & (df['Hour'] <= 8)]

In [ ]:
# Grab all the rows from patient p004893 or p004893
df[(df['PatientID'] == 'p004893') | (df['PatientID'] == 'p017322')]

Note that when we want to combine boolean conditions together, we used the `&` instead of `and` and  `|` instead of `or`. This is because we need to do an element-wise logical comparison between two `Series` of boolean values rather than two boolean values:

In [ ]:
print(df['Hour'] >= 7)
print(df['Hour'] <= 8)

# Creating and Removing DataFrame Columns

Creating a column requires accessing the column and then setting its value with an assignment statement. If you assign a single value, it will be propagated to all of the rows. If you assign as many values as there are rows, `pandas` will put the values in the correct rows. If you assign an arbitrary number of values, your code won't work.

In [ ]:
# Assume a default value
df['Ethnicity'] = 'Caucasian'
df

In [ ]:
# Use an empty placeholder
df['Address'] = np.nan
df

In [ ]:
# Create a column based on the values in another column
df['Age in days'] = df['Age'] * 365
df

To remove columns, you can use the `.drop()` method:

In [ ]:
df2 = df.drop(columns=['Address', 'Age in days'])
df2

As a useful aside, many methods provided by `pandas` have a handy argument called `inplace` that will allow you to modify the original object rather than needing to assign the output to a new variable:

In [ ]:
df.drop(columns=['Address', 'Age in days'],inplace=True)
df

# Creating and Removing DataFrame Rows

The best way to add one or many rows to a `DataFrame` is by using the `.append()` method. Here are two different ways to use it:

In [ ]:
# Add a single patient from a dictionary
# Note: this will eventually be deprecated
new_patient = {'PatientID': 'Alex', 'Age': 31, 'Gender': 1, 'SepsisLabel': 0,
               'Hour': 0, 'HR': 75, 'O2Sat': 98, 'SBP': 96, 'DBP': 60, 'Resp': 18}
df.append(new_patient, ignore_index=True)

In [ ]:
# Add a group of patients from another DataFrame
df2 = df.loc[:5]
df1and2 = pd.concat([df,df2], ignore_index=True)
df1and2

You'll see that in both cases, we set the `ignore_index` argument to `True` so that `pandas` could automatically generate new indices for us. Had we not done this, `pandas` would have either used the indices from the data being added or thrown an error.

You can drop rows by their assigned index lable using the `.drop()` method that we used for columns.

In [ ]:
df1and2.drop(index=[320, 321, 322, 323, 324], inplace=True)

If you want to drop rows that satisfy a set of criteria, you can use the same technique we used earlier to access rows by boolean conditions and then grab the index of the final result.

In [ ]:
# Remove all data from patient p004893
df1and2.drop(index=df1and2[(df1and2['PatientID'] == 'p004893')].index, inplace=True)
df1and2

# Other Operations

Like with `numpy`, `pandas` provides a number of methods and functions you can use to get the most use out of your `Series` and `DataFrames`.

In [ ]:
# Compute the min, max, mean
print(f"Min: {df['Age'].min()}")
print(f"Mean: {df['Age'].mean()}")
print(f"Max: {df['Age'].max()}")

In [ ]:
# Find the positional index of the min and max values
print(f"Argmin: {df['Age'].argmin()}")
print(f"Argmax: {df['Age'].argmax()}")

In [ ]:
# Sorting by age, then by HR value
df.sort_values(by=['Age', 'HR'], inplace=True)
df

In [ ]:
# Get the frequency of each value in a Series
df['PatientID'].value_counts()

`pandas` also has a helpful method called `.describe()` that will compute all of the summary statistics for every column of your `DataFrame` and put those results in a new `DataFrame`.

In [ ]:
df.describe()

As with `numpy` arrays, you can perform element-wise operations on columns.

In [ ]:
df['Age in Days'] = df['Age'] * 365
df

,PatientID,Age,Sex,SepsisLabel,Hour,HR,O2Sat,SBP,DBP,Resp,Age in Days
0,p007806,51.86,0,0,0,NaN,NaN,NaN,NaN,NaN,18928.9
1,p007806,51.86,0,0,1,70.0,98.0,106.0,NaN,15.0,18928.9
2,p007806,51.86,0,0,2,NaN,NaN,NaN,NaN,NaN,18928.9
3,p007806,51.86,0,0,3,63.0,97.0,104.0,NaN,19.0,18928.9
4,p007806,51.86,0,0,4,74.0,99.0,107.0,NaN,20.0,18928.9
...,...,...,...,...,...,...,...,...,...,...,...
371,p005568,76.94,0,0,36,NaN,NaN,NaN,NaN,NaN,28083.1
372,p005568,76.94,0,0,37,78.0,96.0,147.0,NaN,10.0,28083.1
373,p005568,76.94,0,0,38,NaN,NaN,NaN,NaN,NaN,28083.1
374,p005568,76.94,0,0,39,78.0,95.0,125.0,NaN,15.0,28083.1


Some operations may require you to perform element-wise operations that cannot be written for an entire `Series`. Another option for this situations is to use the `apply()` function, which applies a specified function to each element along a `Series` or an axis of a `DataFrame`. A few examples are shown below:

In [ ]:
# Using a regular function on a Series
def sex_to_text(sex):
    if sex == 0:
        return 'Male'
    elif sex == 1:
        return 'Female'
    else:
        return 'Non-binary'

df['Sex Label'] = df['Sex'].apply(sex_to_text)
df

,PatientID,Age,Sex,SepsisLabel,Hour,HR,O2Sat,SBP,DBP,Resp,Age in Days,Sex Label
0,p007806,51.86,0,0,0,NaN,NaN,NaN,NaN,NaN,18928.9,Male
1,p007806,51.86,0,0,1,70.0,98.0,106.0,NaN,15.0,18928.9,Male
2,p007806,51.86,0,0,2,NaN,NaN,NaN,NaN,NaN,18928.9,Male
3,p007806,51.86,0,0,3,63.0,97.0,104.0,NaN,19.0,18928.9,Male
4,p007806,51.86,0,0,4,74.0,99.0,107.0,NaN,20.0,18928.9,Male
...,...,...,...,...,...,...,...,...,...,...,...,...
371,p005568,76.94,0,0,36,NaN,NaN,NaN,NaN,NaN,28083.1,Male
372,p005568,76.94,0,0,37,78.0,96.0,147.0,NaN,10.0,28083.1,Male
373,p005568,76.94,0,0,38,NaN,NaN,NaN,NaN,NaN,28083.1,Male
374,p005568,76.94,0,0,39,78.0,95.0,125.0,NaN,15.0,28083.1,Male


In [ ]:
# Using a lambda function
df['HR Check'] = df['HR'].apply(lambda hr: hr > 60 and hr < 100)
df

,PatientID,Age,Sex,SepsisLabel,Hour,HR,O2Sat,SBP,DBP,Resp,Age in Days,Sex Label,HR Check
0,p007806,51.86,0,0,0,NaN,NaN,NaN,NaN,NaN,18928.9,Male,False
1,p007806,51.86,0,0,1,70.0,98.0,106.0,NaN,15.0,18928.9,Male,True
2,p007806,51.86,0,0,2,NaN,NaN,NaN,NaN,NaN,18928.9,Male,False
3,p007806,51.86,0,0,3,63.0,97.0,104.0,NaN,19.0,18928.9,Male,True
4,p007806,51.86,0,0,4,74.0,99.0,107.0,NaN,20.0,18928.9,Male,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...
371,p005568,76.94,0,0,36,NaN,NaN,NaN,NaN,NaN,28083.1,Male,False
372,p005568,76.94,0,0,37,78.0,96.0,147.0,NaN,10.0,28083.1,Male,True
373,p005568,76.94,0,0,38,NaN,NaN,NaN,NaN,NaN,28083.1,Male,False
374,p005568,76.94,0,0,39,78.0,95.0,125.0,NaN,15.0,28083.1,Male,True


In [ ]:
# Using a lambda function on multiple columns
def check_bp(sbp, dbp):
    return (sbp < 130) and (dbp < 85)

df['BP Check'] = df.apply(lambda row: check_bp(row.SBP, row.DBP), axis=1)
df

,PatientID,Age,Sex,SepsisLabel,Hour,HR,O2Sat,SBP,DBP,Resp,Age in Days,Sex Label,HR Check,BP Check
0,p007806,51.86,0,0,0,NaN,NaN,NaN,NaN,NaN,18928.9,Male,False,False
1,p007806,51.86,0,0,1,70.0,98.0,106.0,NaN,15.0,18928.9,Male,True,False
2,p007806,51.86,0,0,2,NaN,NaN,NaN,NaN,NaN,18928.9,Male,False,False
3,p007806,51.86,0,0,3,63.0,97.0,104.0,NaN,19.0,18928.9,Male,True,False
4,p007806,51.86,0,0,4,74.0,99.0,107.0,NaN,20.0,18928.9,Male,True,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
371,p005568,76.94,0,0,36,NaN,NaN,NaN,NaN,NaN,28083.1,Male,False,False
372,p005568,76.94,0,0,37,78.0,96.0,147.0,NaN,10.0,28083.1,Male,True,False
373,p005568,76.94,0,0,38,NaN,NaN,NaN,NaN,NaN,28083.1,Male,False,False
374,p005568,76.94,0,0,39,78.0,95.0,125.0,NaN,15.0,28083.1,Male,True,False
